In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, KFold

In [2]:
X_full = pd.read_csv('classes_dataset.csv', dtype={col: np.float32 for col in ['lcc', 'lcom*']})

In [3]:
X_full.head()

,Unnamed: 0,dit,fanin,fanout,lcc,lcom*,loc,noc,rfc,innerClassesQty,totalFieldsQty,totalMethodsQty,wmc,godClass,refusedBequest
0,0,4,1,6,0.0,0.592593,39,0,12,0,3,9,10,0,0
1,1,4,2,7,0.0,0.000000,18,0,6,0,0,4,6,0,0
2,2,1,0,0,0.0,1.000000,56,0,0,3,4,4,4,0,0
3,3,1,1,0,0.0,0.000000,4,0,0,0,0,2,2,0,0
4,4,3,3,5,0.0,0.000000,16,0,7,0,0,4,4,0,0


In [4]:
X = X_full.drop(['godClass'], axis=1)
# retirar outro smell
X = X.drop(['refusedBequest'], axis=1)
y = X_full.godClass

In [5]:
X.head()

,Unnamed: 0,dit,fanin,fanout,lcc,lcom*,loc,noc,rfc,innerClassesQty,totalFieldsQty,totalMethodsQty,wmc
0,0,4,1,6,0.0,0.592593,39,0,12,0,3,9,10
1,1,4,2,7,0.0,0.000000,18,0,6,0,0,4,6
2,2,1,0,0,0.0,1.000000,56,0,0,3,4,4,4
3,3,1,1,0,0.0,0.000000,4,0,0,0,0,2,2
4,4,3,3,5,0.0,0.000000,16,0,7,0,0,4,4


In [6]:
#avaliar com calma
X.fillna(0, inplace=True)

In [7]:
np.any(np.isnan(X))

False

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50930 entries, 0 to 50929
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       50930 non-null  int64  
 1   dit              50930 non-null  int64  
 2   fanin            50930 non-null  int64  
 3   fanout           50930 non-null  int64  
 4   lcc              50930 non-null  float32
 5   lcom*            50930 non-null  float32
 6   loc              50930 non-null  int64  
 7   noc              50930 non-null  int64  
 8   rfc              50930 non-null  int64  
 9   innerClassesQty  50930 non-null  int64  
 10  totalFieldsQty   50930 non-null  int64  
 11  totalMethodsQty  50930 non-null  int64  
 12  wmc              50930 non-null  int64  
dtypes: float32(2), int64(11)
memory usage: 4.7 MB


In [9]:
## PIPELINE

In [10]:
SEED=0
decision_tree_regressor = DecisionTreeRegressor(random_state=SEED, criterion='mse')

In [11]:
pipe = Pipeline(steps=[
            ('minmaxscaler', MinMaxScaler()),
            ('stdscaler',  StandardScaler()),
            ('regressor', decision_tree_regressor)
    ])

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=3)

In [13]:
# https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/
parameters = dict(
    regressor__max_depth= [2,3,5,7,9,11,13],
    regressor__min_samples_split= [32,64,128,256],
    regressor__min_samples_leaf= [32,64,128,256])

In [14]:
randomized_search = RandomizedSearchCV(pipe,
                              parameters,
                              cv = KFold(n_splits=5),
                                        n_iter=20,
                                      random_state=11)

In [15]:
# Exporta precisão? Acurácia? Etc...
randomized_search.fit(X, y)

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                                             ('stdscaler', StandardScaler()),
                                             ('regressor',
                                              DecisionTreeRegressor(random_state=0))]),
                   n_iter=20,
                   param_distributions={'regressor__max_depth': [2, 3, 5, 7, 9,
                                                                 11, 13],
                                        'regressor__min_samples_leaf': [32, 64,
                                                                        128,
                                                                        256],
                                        'regressor__min_samples_split': [32, 64,
                                                                         128,
                                           

In [16]:
resultados = pd.DataFrame(randomized_search.cv_results_)
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__min_samples_split,param_regressor__min_samples_leaf,param_regressor__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.073102,0.011474,0.003125,0.006250,64,128,2,"{'regressor__min_samples_split': 64, 'regresso...",0.466120,0.333688,0.623443,0.434700,0.549636,0.481517,0.099061,20
1,0.110947,0.006399,0.006774,0.008338,128,128,11,"{'regressor__min_samples_split': 128, 'regress...",0.439621,0.405572,0.599163,0.555916,0.583569,0.516768,0.078867,16
2,0.126732,0.014839,0.006250,0.007655,256,64,11,"{'regressor__min_samples_split': 256, 'regress...",0.466343,0.525677,0.713138,0.662148,0.611701,0.595801,0.089554,6
3,0.129161,0.010911,0.004928,0.005795,128,64,13,"{'regressor__min_samples_split': 128, 'regress...",0.458668,0.527436,0.724442,0.644048,0.628752,0.596669,0.093207,3
4,0.066414,0.003191,0.000000,0.000000,32,32,2,"{'regressor__min_samples_split': 32, 'regresso...",0.466866,0.333970,0.623443,0.434700,0.549636,0.481723,0.098954,19


In [17]:
# qual acuracia e precisao?
randomized_search.best_params_

{'regressor__min_samples_split': 256,
 'regressor__min_samples_leaf': 32,
 'regressor__max_depth': 7}

In [19]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [24]:
precision_score(y, y)

1.0

In [25]:
recall_score(y,y)

1.0

In [26]:
accuracy_score(y,y)

1.0

In [27]:
f1_score(y,y)

1.0